## HotChili Analytics trading notebook template
#### Configure by setting ALGO_NAME in cell below.
#### Run various options (backtest, ingest, live) by uncommenting one cell 

In [1]:
%matplotlib inline
%load_ext zipline

# %reload_ext zipline # Uncomment and use this when already loaded zipline extension for magic cell usage.

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))


In [2]:
import pandas as pd

# Options you can uncomment and set:

# pd.set_option("max_colwidth", 300)
# pd.set_option("display.max_rows", 300)
# pd.set_option("display.max_columns", 50)
# pd.set_option('precision', 2)
# pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
import os

hca_root_path = os.environ['HCA_ROOT']
print(f"hca_root_path = {hca_root_path}")

hca_root_path = /home/hca-ws2004/hca


# Construct algorithm strategy path names

Assumptions:

- the strategy is in a directory with the same name as the strategy in `ALGO_NAME` below
- the strategy is located in the hca-resources directory, which is located relative to `hca_root_path`, found above

In [4]:
ALGO_NAME = "HCA_Trendfollowing" # <--- Supply name here

Other variables are derived from `ALGO_NAME`:

In [5]:
HCA_RESOURCES_PATH = hca_root_path + "/hca-resources/" 
ALGO_PATH          = HCA_RESOURCES_PATH + ALGO_NAME + "/" 
ALGO_BT            = ALGO_PATH + ALGO_NAME + ".py"
ALGO_BT_OUT        = ALGO_PATH + ALGO_NAME + ".pkl"
ALGO_LIVE          = ALGO_PATH + ALGO_NAME + "_Live" + ".py"

print(f"""
ALGO_NAME          = {ALGO_NAME}
HCA_RESOURCES_PATH = {HCA_RESOURCES_PATH}
ALGO_PATH          = {ALGO_PATH}
ALGO_BT            = {ALGO_BT}
ALGO_BT_OUT        = {ALGO_BT_OUT}
ALGO_LIVE          = {ALGO_LIVE}

Contents of algo directory:
""")

!ls $ALGO_PATH


ALGO_NAME          = HCA_Trendfollowing
HCA_RESOURCES_PATH = /home/hca-ws2004/hca/hca-resources/
ALGO_PATH          = /home/hca-ws2004/hca/hca-resources/HCA_Trendfollowing/
ALGO_BT            = /home/hca-ws2004/hca/hca-resources/HCA_Trendfollowing/HCA_Trendfollowing.py
ALGO_BT_OUT        = /home/hca-ws2004/hca/hca-resources/HCA_Trendfollowing/HCA_Trendfollowing.pkl
ALGO_LIVE          = /home/hca-ws2004/hca/hca-resources/HCA_Trendfollowing/HCA_Trendfollowing_Live.py

Contents of algo directory:

HCA_Trendfollowing.ipynb    HCA_Trendfollowing.py
HCA_Trendfollowing_Live.py  HCA_Trendfollowing_tearsheet.ipynb
HCA_Trendfollowing.pkl


## Zipline backtest: 

- Method: Jupyter magic cell (%%) 
- Execution of zipline code in cell containing command line command
- Uncomment first line and hit (shift-enter) inside the cell to run simulation backtest

In [6]:
# %%zipline --start=2018-1-1 --end=2020-8-10 -b sharadar-funds -o $ALGO_BT_OUT

# Source: adapted from various algos on quantopian
# HCA Conversion Date: 08-14-2020
# Conversion Author: Anthony garner

# Simple trend following portfolio


import matplotlib.pyplot as plt
import pandas as pd
import logging as log

from zipline.api import (order, cancel_order, get_open_orders, symbol, symbols, 
                         date_rules, time_rules, order_target_percent, 
                         record, schedule_function, get_datetime)
from trading_calendars import get_calendar


def initialize(context):
    schedule_function(func=trade, date_rule=date_rules.every_day(),
                      time_rule=time_rules.market_open(),half_days=True)
    schedule_function(func=cancel,time_rule=time_rules.market_close(minutes=5),  
                      date_rule=date_rules.every_day(),half_days=True)  
    schedule_function(func=reorder, time_rule=time_rules.market_open(minutes=5),  
                      date_rule=date_rules.every_day(),half_days=True)
    context.assets = symbols('SPY')
    context.bonds = symbol('SHY') 
    context.rebalance_date = 0
    context.fired = False
    context.rebalance_inteval = 'M'#'Q', #'D', #'M' #'Q' #'Y'
    context.top_n_by_momentum = pd.Series()  
    #Choose X stocks out of portfolio of Y stocks- how many stocks to hold - top X by momentum 
    context.stocks=1
    #Lookback for momentum calculation
    context.momentum_days=60
    #set at less than 1 to ensure no leverage
    context.leverage_buffer=0.99
    #Set to 0 to reject any stocks with negative momentum, set to -1 to accept stocks with negative momentum
    context.trend =0.0
    context.reorder_dict = {}  


def handle_data(context, data):
    record(SPY=data[symbol('SPY')].price)


def is_new_day(context, now):
    return ( (now.year > context.rebalance_date.year) or (now.month > context.rebalance_date.month) or((now.month == context.rebalance_date.month) and (now.day > context.rebalance_date.day)))             


def is_new_month(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month > context.rebalance_date.month)))


def is_new_quarter(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month == context.rebalance_date.month + 3)))


def is_new_year(context, now):
    return (now.year > context.rebalance_date.year)


def need_rebalance(context, now):
    return ((context.rebalance_inteval == 'Y' and is_new_year(context, now))or 
           (context.rebalance_inteval == 'Q' and is_new_quarter(context, now)) or 
           (context.rebalance_inteval == 'M' and is_new_month(context, now)) or 
           (context.rebalance_inteval == 'D' and is_new_day(context, now)))


def compute_momentum(context,data):  
    price_history = data.history(context.assets, "price", context.momentum_days+5, "1d")
    momentum = price_history.pct_change(context.momentum_days).iloc[-1]
    #for index,value in momentum.items():
        #log.debug("unfiltered momentun"+" "+ str(index)+" "+ str(value) )
    context.top_n_by_momentum = momentum.nlargest(context.stocks).where(momentum>context.trend).dropna()
    #for index,value in context.top_n_by_momentum.items():
        #log.debug("context.top_n_by_momentun"+" "+str(index)+" " + str(value) )
    return context.top_n_by_momentum


def init_portfolio(context, data):
    weights=0.0
    reserve_allocation=0.0
    compute_momentum(context, data)
    for index,value in context.top_n_by_momentum.items():
        if data.can_trade(index):
            weights =weights +1/context.stocks
            order_target_percent(index, (1/context.stocks)* context.leverage_buffer)
    #Assign weighting and an order to the reserve asset if and when appropriate
    if weights <1 and data.can_trade(context.bonds):
        reserve_allocation=1-weights
        order_target_percent(context.bonds, reserve_allocation* context.leverage_buffer)     


def rebalance(context, data):
    weights=0.0
    reserve_allocation=0.0
    compute_momentum(context, data)
    for x in context.portfolio.positions:
        if (x not in context.top_n_by_momentum and x != context.bonds):
            order_target_percent(x, 0)            
    
    for index,value in context.top_n_by_momentum.items():
        if data.can_trade(index)and index != context.bonds:
            weights =weights +1/context.stocks
            order_target_percent(index, (1/context.stocks)*context.leverage_buffer)
        elif data.can_trade(index) and index != context.bonds: 
            order_target_percent(index, 0)
    #Assign weighting and an order to the reserve asset if and when appropriate        
    if data.can_trade(context.bonds):
        reserve_allocation=1-weights
        order_target_percent(context.bonds, reserve_allocation* context.leverage_buffer)        


# Will be called daily. 
def trade(context, data):
    if not context.fired:
        context.rebalance_date = get_datetime()
        log.info("build portfolio at " + str(context.rebalance_date))
        init_portfolio(context, data)
        context.fired = True
        now = get_datetime()
    else:
        now = get_datetime()
        if (need_rebalance(context, now)):
            log.info("new rebalance arrivied:" + str(now))
            rebalance(context, data)
            context.rebalance_date = now

# orphan:
# open_orders = get_all_open_orders()  
# for order in open_orders:  
# log.info("Rebalance Order {0:s} for {1:,d} shares" .format(order.sid.symbol,order.amount))


# Called Daily to replace/re-order partially or unfilled orders
def cancel(context, data):  
    open_orders = get_all_open_orders()  
    for order in open_orders:  
        log.info("X CANCELED {0:s} with {1:,d} / {2:,d} filled" 
            .format(order.sid.symbol,  
                    order.filled,  
                    order.amount))  
        cancel_order(order)  
        context.reorder_dict[order.sid] = order.amount - order.filled


def get_all_open_orders():  
    from itertools import chain  
    orders = chain.from_iterable(get_open_orders().values())  
    return list(orders)  


# Called Daily to replace/re-order partially or unfilled orders
def reorder(context, data):  
    for stock, amount in context.reorder_dict.items():  
        order(stock, amount)  
        log.info("Reorder {stock} {amount}".format(stock=stock, amount=amount))  
    context.reorder_dict = {}


def analyze(context, perf):
    ax1 = plt.subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax2 = plt.subplot(212, sharex=ax1)
    perf.SPY.plot(ax=ax2)
    plt.gcf().set_size_inches(18, 8)
    plt.show()

## Display your current bundles

In [7]:
#!zipline bundles # Finds all bundles

## Ingest Bundle funds assets. 

In [8]:
# Ingest Sharadar funds assets for today, if needed.

# Only need to ingest Funds for this algo, and this takes less processing time and system memory than ingesting
# all of Sharadar Equities plus Funds bundle (sharadar-eqfd)

# Yahoo Direct Data Bundle Ingest
# Quick way
# !export YAHOO_SYM_LST=SPY,ZSL,KOLD,GLD,SHY,XLP,IEF,TLT,QQQ;zipline ingest -b yahoo_direct

# Long way - Set environ variable, so can use it later after bundle is created to find out what is in the bundle.
os.environ["YAHOO_SYM_LST"] = "SPY,ZSL,KOLD,GLD,SHY,XLP,IEF,TLT,QQQ"
!zipline ingest -b yahoo_direct
yahoo_direct_sym_lst = os.environ['YAHOO_SYM_LST']
print(f"yahoo_direct_sym_lst = {yahoo_direct_sym_lst}")

# !zipline ingest -b sharadar-funds 
##### !zipline ingest -b hca-symbol


/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
extension: hca_root_path = /home/hca-ws2004/hca
extension:TODAY_STR = 2021-04-10
extension:TWO_YR_AGO_STR = 2019-04-10
extension:  start_date=2019-04-10 end_date = 2021-04-10
[2021-04-11 03:49:29.486586] INFO: zipline.data.bundles.ingester: symbols are: ('TLT', 'IEF', 'ZSL', 'SHY', 'SPY', 'QQQ', 'KOLD', 'GLD', 'XLP')
[2021-04-11 03:49:29.487428] INFO: zipline.data.bun

## Zipline backtest, alternative method

- Method: command line
- Execution of zipline code, located in a file, using below command line execution with magic (`!`) invocation
- This line can also be run in a terminal by copying everything past the `!` and pasting (shift-insert) it into the target terminal

In [9]:
!zipline run -f $ALGO_BT  --start=2018-1-1 --end=2020-10-02 --capital-base 10000 -b sharadar-funds -o $ALGO_BT_OUT

/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-r2-001/miniconda3/envs/hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
extension: hca_root_path = /home/hca-ws2004/hca
extension:TODAY_STR = 2021-04-10
extension:TWO_YR_AGO_STR = 2019-04-10
extension:  start_date=2019-04-10 end_date = 2021-04-10
[2021-04-11 03:49:34.951572] WARNING: Loader: Refusing to download new benchmark data because a download succeeded at 2021-04-11 03:17:05.084462+00:00.
[2021-04-11 03:49:34.984385] WARNING: Loade

## Run Zipline live on IB-TWS via command line

- Method: command line
- Execution of zipline code using below command line execution using magic (`!`) invocation

**Notes:** 
- IB-TWS or IB-Gateway must be running, with `IB_ACCT` and `IB_URI` port being correct to live trade
- Change `I_WANT_TO_RUN_THIS_CODE` to `True` below to run zipline live on IB-TWS/IB-Gateway

In [10]:
TODAY = pd.datetime.today().strftime("%Y-%m-%d")
print("TODAY = {}".format(TODAY))

TODAY = 2021-04-10


In [11]:
ALGO_STATE = ALGO_PATH + "strategy.state" 
ALGO_RTB   = ALGO_PATH + "realtime-bars/"

# Edit the following URI to match your IB account and port info.
IB_ACCT = "DU1568488"
IB_URI = "127.0.0.1:7497:1301"
BUNDLE = 'yahoo_direct'
#BUNDLE = 'hca_symbol'

# Change following to 'True' and run cell (control-enter) to execute live run.
I_WANT_TO_RUN_THIS_CODE = False
# I_WANT_TO_RUN_THIS_CODE = True

if I_WANT_TO_RUN_THIS_CODE:
    
    !zipline run \
        -s $TODAY \
        -f $ALGO_LIVE \
        --bundle $BUNDLE \
        --broker ib \
        --broker-uri $IB_URI \
        --broker-acct $IB_ACCT \
        --data-frequency daily \
        --state-file $ALGO_STATE \
        --realtime-bar-target $ALGO_RTB 